# This is notebook for extraction of the ON/OFF and expected counts (using the corresponding python script) from the datasets considering different models of photon-ALP mixing in the ALPs parameter space.

### It was written by Giacomo D'Amico and Ivana Batković for the needs of the article "Constraints on axion-like particles with the Perseus Galaxy Cluster with MAGIC". 

### If you wish to use the script and reproduce the results, you are invited to contact the authors:

### Giacomo D'Amico, giacomo.damico@uib.no
### Ivana Batković, ivana.batkovic@phd.unipd.it


### For running this script, gammapy-0.20 version is needed. In case you miss it, check: https://docs.gammapy.org/0.20/

# LOAD MODULES

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('xtick', labelsize=20)   
plt.rc('ytick', labelsize=20)
plt.rc('text', usetex=True)
plt.rc('font', family='serif',size=25)

import numpy as np
import astropy.units as u
from pathlib import Path
import glob
from astropy.io import fits
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion, PointSkyRegion

import pickle
import gzip

from scipy.stats import chi2
from functools import partial
import scipy.special as scipys

# GAMMAPY MODULES
### gammapy 0.20

In [ ]:
from gammapy.modeling import Fit
import gammapy.irf as irf
from gammapy.irf import load_cta_irfs
from gammapy.data import Observation, Observations, DataStore
from gammapy.utils.random import get_random_state
from gammapy.maps import MapAxis

# models modules
from gammapy.modeling.models import (
    Model,
    Models,
    SkyModel,
    PowerLawSpectralModel,
    PowerLawNormSpectralModel,
    ExpCutoffPowerLawSpectralModel,
    PointSpatialModel,
    GaussianSpatialModel,
    TemplateSpatialModel,
    FoVBackgroundModel,
    SpectralModel,
    TemplateSpectralModel,
    EBLAbsorptionNormSpectralModel,

)
# dataset modules
from gammapy.datasets import (
    MapDataset, 
    MapDatasetOnOff, 
    MapDatasetEventSampler,
    SpectrumDatasetOnOff,
    SpectrumDataset, 
    Datasets,
    FluxPointsDataset
)

from gammapy.maps import MapAxis, WcsGeom, Map, RegionGeom
from gammapy.makers import MapDatasetMaker, SpectrumDatasetMaker, ReflectedRegionsBackgroundMaker, WobbleRegionsFinder
from gammapy.estimators import FluxPointsEstimator

from gammapy.stats.fit_statistics import wstat

# LOAD ALPGrid class

In [ ]:
from expected_counts import compute_expected_counts, get_array_from_value_and_error, check_keys_in_dict

In [ ]:
observations_flare = Observations()
for filename in glob.glob(f"../path/where/the/fits/files/are/stored/flaring_state/*fits"):
    observations_flare.append(Observation.read(filename))

observations_post_flare = Observations()
for filename in glob.glob(f"../path/where/the/fits/files/are/stored/post-flaring_state/*fits"):
    observations_post_flare.append(Observation.read(filename)) 

observations_low_state = Observations()
for filename in glob.glob(f"../path/where/the/fits/files/are/stored/low-state/*fits"):
    observations_low_state.append(Observation.read(filename)) 


In [ ]:
print( len(observations_flare))
print( len(observations_post_flare))
print( len(observations_low_state))

### Function for computing ON/OFF counts from datasets

In [ ]:
def get_Non_Noff_from_obs_list( observations, bkg_maker, on_center, energy_axis ):
    
    geom             = RegionGeom.create(region=on_center, axes=[energy_axis])
    dataset_empty    = SpectrumDataset.create(geom=geom)
    dataset_maker    = SpectrumDatasetMaker(containment_correction=False, selection=["counts"])
    
    n_on  = []
    n_off = []
    for obs in observations:
        dataset                 = dataset_maker.run(dataset_empty.copy(name=f"{obs.obs_id}"), obs)
        dataset.mask_fit        = dataset.counts.geom.energy_mask(emin, emax)
        dataset                 = bkg_maker.run(dataset, obs)
        dataset.mask_fit        = dataset.counts.geom.energy_mask(emin, emax)
        #datasets.append(dataset)
        n_on.append( dataset.counts.data.flatten())
        n_off.append( dataset.counts_off.data.flatten())
    
    n_on  = np.array( n_on )
    n_off = np.array( n_off )
        
    return np.sum(n_on, axis=0,dtype=int), np.sum(n_off, axis=0,dtype=int)

In [ ]:
# s --> (  B-field, energy, SEDpar )
# n --> ( simulations, energy

# DEFINE ENERGY GEOMETRY for each DATASE

In [ ]:
nbins_true = 200

# FLARE
emin             = 50*u.GeV
emax             = 2.1*u.TeV
nbins            = 27     
en_edges               = np.geomspace(  emin, emax, nbins) 
nergy_axis             = MapAxis.from_edges(en_edges, interp='log' , unit="GeV", name="energy")
energy_reco_axis_flare = MapAxis.from_edges(en_edges, interp='log' , unit="GeV", name="energy")
energy_true_axis_flare = MapAxis.from_energy_bounds(5, 5e4, nbin=nbins_true, per_decade=False, unit="GeV", name="energy_true")


# POST FLARE
emin             = 64*u.GeV
emax             = 1.4*u.TeV
nbins            = 25    
en_edges                    = np.geomspace(  emin, emax, nbins) 
nergy_axis                  = MapAxis.from_edges(en_edges, interp='log' , unit="GeV", name="energy")
energy_reco_axis_post_flare = MapAxis.from_edges(en_edges, interp='log' , unit="GeV", name="energy")
energy_true_axis_post_flare = MapAxis.from_energy_bounds(5, 5e4, nbin=nbins_true, per_decade=False, unit="GeV", name="energy_true")

# LOW STATE
emin             = 70*u.GeV
emax             = 2.1*u.TeV
nbins            = 20     
en_edges                   = np.geomspace(  emin, emax, nbins) 
nergy_axis                 = MapAxis.from_edges(en_edges, interp='log' , unit="GeV", name="energy")
energy_reco_axis_low_state = MapAxis.from_edges(en_edges, interp='log' , unit="GeV", name="energy")
energy_true_axis_low_state = MapAxis.from_energy_bounds(5, 5e4, nbin=nbins_true, per_decade=False, unit="GeV", name="energy_true")

### Background

In [ ]:
#Number of OFF regions to be used to estimate the background
n_off_regions = 3
alpha         = 1/n_off_regions
region_finder = WobbleRegionsFinder(n_off_regions=n_off_regions)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

### ON REGION

In [ ]:
#ON REGION
source_coordinates_ngc1275 = SkyCoord.from_name("NGC1275")
on_center_ngc1275          = PointSkyRegion(source_coordinates_ngc1275)

# GET N_ON and N_OFF counts per each bin per each DATASET

In [ ]:
%%time

#datasets_flare = get_datasets_from_obs_list
n_on_flare, n_off_flare = get_Non_Noff_from_obs_list( observations=observations_flare, on_center=on_center_ngc1275,
                                                bkg_maker=bkg_maker,
                                                energy_axis=energy_reco_axis_flare)
#datasets_post_flare = get_datasets_from_obs_list(
n_on_post_flare, n_off_post_flare = get_Non_Noff_from_obs_list(  observations=observations_post_flare,on_center=on_center_ngc1275,
                                                bkg_maker=bkg_maker,
                                                energy_axis=energy_reco_axis_post_flare)

# datasets_low_state = get_datasets_from_obs_list(
n_on_low_state, n_off_low_state = get_Non_Noff_from_obs_list(  observations=observations_low_state,on_center=on_center_ngc1275,
                                                bkg_maker=bkg_maker,
                                                energy_axis=energy_reco_axis_low_state)

#datasets_ic310    = get_datasets_from_obs_list( 
n_on_ic310, n_off_ic310 = get_Non_Noff_from_obs_list( observations=observations_ic310,on_center=on_center_ic310,
                                                bkg_maker=bkg_maker,
                                                energy_axis=energy_reco_axis_ic310)


n_on =  [ n_on_flare,       n_on_post_flare,        n_on_low_state,         n_on_ic310]
n_off = [ n_off_flare,       n_off_post_flare,        n_off_low_state,         n_off_ic310]



file_name = "./observed_on_counts.pkl"
with open(file_name, 'wb') as f:
    pickle.dump( n_on, f)

file_name =  "./observed_off_counts.pkl"
with open(file_name, 'wb') as f:
    pickle.dump(n_off,f)

# SIMULATE N_ON and N_OFF counts

In [ ]:
def get_exp_signal_counts_from_obs_list( true_flux, observations, source_coord, ereco,etrue, delta_ereco, delta_etrue):
    
    delta_etrue = delta_etrue[None,None,:] 
    delta_ereco = delta_ereco[None,:]
    
    IRF  = np.zeros( (len(ereco),len(etrue)) ) * u.m**2 * u.s/u.GeV
    for obs in observations:
        true_offset = obs.pointing_radec.separation( source_coord)
        IRF        += get_IRF( obs, true_offset, ereco,etrue)
               
    obs_flux           = np.sum( true_flux[:,None,:]*IRF[None,:,:]*delta_etrue,axis=2)
    obs_flux           = obs_flux.to(1/( u.GeV  ))
    exp_signal_counts  = obs_flux*delta_ereco
        

    return np.array( exp_signal_counts.to('').value)# dtype=np.float16)


In [ ]:
from expected_counts import get_IRF

In [ ]:
dominguez = EBLAbsorptionNormSpectralModel.read_builtin("dominguez", redshift=0.01790)

In [ ]:
# GET ABSORPTION
list_of_names_gm = []
path_name = "/path/where/your/npy/files/from/ALPs/absorption/are/stored" 
for name in glob.glob(path_name+"/*.npy"):  
    i_seed = 0
    np.random.seed(i_seed)
    
    #i.e.
    #name =  '../ngc1275_100real_m_464_g_229_.npy'
    
    names_split = str.split(name, "_")
    g = names_split[-2]
    m = names_split[-4]
    g = float(g) * 1e-14 / u.GeV
    m = float(m) * 1e-2 * u.neV
    
        
    total_number_of_simulations = 100
    number_of_B_field_realizations = 100
    list_of_B_field = np.arange(number_of_B_field_realizations)
    
    
    # FLARE
    np.random.seed(i_seed)
    p_gamma_gamma = []
    for i in list_of_B_field:
        en_absorp_array     = np.load(name)
        energy              = en_absorp_array[0] * u.GeV
        values              = en_absorp_array[1+i] * u.Unit("")
        absorption          = TemplateSpectralModel(energy, values)
        p_gamma_gamma.append( absorption( energy_true_axis_flare.center).to('').value )
    p_gamma_gamma       = np.array(p_gamma_gamma)
    
    reference =0.3 * u.TeV
    amplitude = 16.1e-10 * u.Unit("TeV-1 cm-2 s-1")
    index     = 2.11
    lambda_   = 1.24 * u.Unit("TeV-1")
    energies  = energy_true_axis_flare.center
    true_flux = amplitude*(energies/reference)**(-index) * np.exp(-energies*lambda_) 
    true_flux = true_flux[None,:] * p_gamma_gamma
    exp_signals_flare = get_exp_signal_counts_from_obs_list( 
                                        true_flux    = true_flux, 
                                        observations = observations_flare, 
                                        source_coord = source_coordinates_ngc1275, 
                                        ereco        = energy_reco_axis_flare.center,  
                                        etrue        = energy_true_axis_flare.center, 
                                        delta_ereco  = energy_reco_axis_flare.bin_width,
                                        delta_etrue  = energy_true_axis_flare.bin_width,
                                                     )
    
    # SIMULATE N_ON and N_OFF from the expected signal counts
    np.random.seed(i_seed)
    n_signal          =  np.random.poisson( exp_signals_flare) 
    assumed_b         =  np.reshape( np.repeat( n_off_flare, total_number_of_simulations), (len(n_off_flare),total_number_of_simulations)).T
    np.random.seed(i_seed)
    n_b_on            =  np.random.poisson(assumed_b/3) ### 3 is the number of OFF regions used to estimate the background
    n_on_fake_flare   =  n_signal + n_b_on
    np.random.seed(i_seed)
    n_off_fake_flare  =  np.random.poisson(assumed_b)
    
    
    
    
    # POST FLARE
    np.random.seed(i_seed)
    p_gamma_gamma = []
    for i in list_of_B_field:
        en_absorp_array     = np.load(name)
        energy              = en_absorp_array[0] * u.GeV
        values              = en_absorp_array[1+i] * u.Unit("") 
        absorption          = TemplateSpectralModel(energy, values)
        p_gamma_gamma.append( absorption( energy_true_axis_post_flare.center).to('').value )
    p_gamma_gamma       = np.array(p_gamma_gamma)
    
    reference = 0.3 * u.TeV
    amplitude = 11.4e-10 * u.Unit("TeV-1 cm-2 s-1")
    index     = 1.79
    lambda_   = 3.45 * u.Unit("TeV-1")
    energies  = energy_true_axis_post_flare.center
    true_flux = amplitude*(energies/reference)**(-index) * np.exp(-energies*lambda_) 
    true_flux = true_flux[None,:] * p_gamma_gamma
    exp_signals_post_flare = get_exp_signal_counts_from_obs_list( 
                                        true_flux    = true_flux, 
                                        observations = observations_post_flare, 
                                        source_coord = source_coordinates_ngc1275, 
                                        ereco        = energy_reco_axis_post_flare.center,  
                                        etrue        = energy_true_axis_post_flare.center, 
                                        delta_ereco  = energy_reco_axis_post_flare.bin_width,
                                        delta_etrue  = energy_true_axis_post_flare.bin_width,
                                                     )
    # SIMULATE N_ON and N_OFF from the expected signal counts
    np.random.seed(i_seed)
    n_signal               =  np.random.poisson( exp_signals_post_flare) 
    assumed_b              =  np.reshape( np.repeat( n_off_post_flare, total_number_of_simulations), (len(n_off_post_flare),total_number_of_simulations)).T
    np.random.seed(i_seed)
    n_b_on                 =  np.random.poisson(assumed_b/3)
    n_on_fake_post_flare   =  n_signal + n_b_on
    np.random.seed(i_seed)
    n_off_fake_post_flare  =  np.random.poisson(assumed_b)
    
    
    
    
    # LOW STATE
    np.random.seed(i_seed)
    p_gamma_gamma = []
    for i in list_of_B_field:
        en_absorp_array     = np.load(name)
        energy              = en_absorp_array[0] * u.GeV
        values              = en_absorp_array[1+i] * u.Unit("") # whatever numer from 1 to 100
        absorption          = TemplateSpectralModel(energy, values)
        p_gamma_gamma.append( absorption( energy_true_axis_low_state.center).to('').value )
    p_gamma_gamma       = np.array(p_gamma_gamma)
    
    reference = 0.3 * u.TeV
    amplitude = 1.1e-10 * u.Unit("TeV-1 cm-2 s-1")
    index     = 2.54
    lambda_   = 2 * u.Unit("TeV-1")
    energies  = energy_true_axis_low_state.center
    true_flux = amplitude*(energies/reference)**(-index) * np.exp(-energies*lambda_) 
    true_flux = true_flux[None,:] * p_gamma_gamma
    exp_signals_low_state = get_exp_signal_counts_from_obs_list( 
                                        true_flux    = true_flux, 
                                        observations = observations_low_state, 
                                        source_coord = source_coordinates_ngc1275, 
                                        ereco        = energy_reco_axis_low_state.center,  
                                        etrue        = energy_true_axis_low_state.center, 
                                        delta_ereco  = energy_reco_axis_low_state.bin_width,
                                        delta_etrue  = energy_true_axis_low_state.bin_width,
                                                     )
    # SIMULATE N_ON and N_OFF from the expected signal counts
    np.random.seed(i_seed)
    n_signal              =  np.random.poisson( exp_signals_low_state) 
    assumed_b             =  np.reshape( np.repeat( n_off_low_state, total_number_of_simulations), (len(n_off_low_state),total_number_of_simulations)).T
    np.random.seed(i_seed)
    n_b_on                =  np.random.poisson(assumed_b/3)
    n_on_fake_low_state   =  n_signal + n_b_on
    np.random.seed(i_seed)
    n_off_fake_low_state  =  np.random.poisson(assumed_b)
    
    
    
    #################################################
    
    
    n_on = []
    n_off = []
    for i in range(100):
        n_on.append( [ n_on_fake_flare[i],       n_on_fake_post_flare[i],        n_on_fake_low_state[i]])
        n_off.append( [ n_off_fake_flare[i],       n_off_fake_post_flare[i],        n_off_fake_low_state[i]])
    name_m = str(int(m.to(0.01*u.neV).value))
    name_g = str(int(g.to( 1e-14 /u.GeV).value))

    ### Prior to this step, create the folder for storing the files with ON and OFF counts, i.e. "fake_on_off_counts"
    
    file_name = "./fake_on_off_counts/fake_on_counts_"+name_m+"__"+name_g+"_.pkl"
    with open(file_name, 'wb') as f:
        pickle.dump( n_on, f)
        
    file_name = "./fake_on_off_counts/fake_off_counts_"+name_m+"__"+name_g+"_.pkl"
    with open(file_name, 'wb') as f:
        pickle.dump(n_off,f)

# GET THE EXPECTED SIGNAL COUNTS per each bin and per each DATASET

In [ ]:
%%time
i = 0


path_name = "/path/where/your/npy/files/from/ALPs/absorption/are/stored" 

    
for name in glob.glob(path_name+"/*.npy"): 
    ## GET INFO ON M AND G FROM FILE NAME
    names_split = str.split(name, "_")
    
    g = names_split[-2]
    m = names_split[-4]
    g = float(g) * 1e-14 / u.GeV
    m = float(m) * 1e-2 * u.neV
    
        
    print(i)
    
    
total_s = compute_expected_counts(name)

### Prior to this step, create the folder for storing the files with expected counts, i.e. "expected_counts"

    for k in range(4):
        name_m = str(int(m.to(0.01*u.neV).value))
        name_g = str(int(g.to( 1e-14 /u.GeV).value))
        file_name = "./expected_counts/expected_counts_"+name_m+"__"+name_g+"_"+str(k)+"_array.npy.gz"

        with gzip.open(file_name, 'wb') as f:
            np.save(f, total_s[k])
            
    i +=1